In [1]:
#THESE SCRIPTS TAKE US FROM RAW DATA TO THE TABLES WE HAVE IN OUR SCHEMA - SEE movie_db_model.png

c:\python37\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [1]:
import pandas as pd
import numpy as np
from itertools import chain

c:\python37\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
##BEGIN CREATION OF DIRECTOR_LOOKUP AND WRITER_LOOKUP TABLES
df_title_crew = pd.read_csv('raw_data/title_crew.tsv', sep='\t')
print(list(df_title_crew))

##Bring in titles
df_title = pd.read_csv('app_data/title.csv')
title_ids = pd.Series(df_title['title_id'])

##Bring in directors
df_director = pd.read_csv('app_data/director.csv')
director_ids = pd.Series(df_director['director_id'])

##Bring in writers
df_writer = pd.read_csv('app_data/writer.csv')
writer_ids = pd.Series(df_writer['writer_id'])

['tconst', 'directors', 'writers']


In [3]:
#one row for each comma separated cell value, inspired by:
#https://stackoverflow.com/questions/50731229/split-cell-into-multiple-rows-in-pandas-dataframe

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lensDirector = df_title_crew['directors'].str.split(',').map(len)

lensWriter = df_title_crew['writers'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
                                #repeating columns
directors_flat = pd.DataFrame({'tconst': np.repeat(df_title_crew['tconst'], lensDirector),
                    #newly split column
                    'directors': chainer(df_title_crew['directors'])})

writers_flat = pd.DataFrame({'tconst': np.repeat(df_title_crew['tconst'], lensWriter),
                    'writers': chainer(df_title_crew['writers'])})


print(directors_flat[:10])
print("------------------")
print(writers_flat[:10])



      tconst  directors
0  tt0000001  nm0005690
1  tt0000002  nm0721526
2  tt0000003  nm0721526
3  tt0000004  nm0721526
4  tt0000005  nm0005690
5  tt0000006  nm0005690
6  tt0000007  nm0005690
6  tt0000007  nm0374658
7  tt0000008  nm0005690
8  tt0000009  nm0085156
------------------
      tconst    writers
0  tt0000001         \N
1  tt0000002         \N
2  tt0000003         \N
3  tt0000004         \N
4  tt0000005         \N
5  tt0000006         \N
6  tt0000007         \N
7  tt0000008         \N
8  tt0000009  nm0085156
9  tt0000010         \N


In [4]:
#Add primary key columns - director
df_director_lookup = directors_flat
df_director_lookup = df_director_lookup.reset_index()
df_director_lookup = df_director_lookup.reset_index()


#Limit to only titles of interest
df_director_lookup = df_director_lookup.loc[df_director_lookup['tconst'].isin(title_ids)]

#Limit to only directors of interest
df_director_lookup = df_director_lookup.loc[df_director_lookup['directors'].isin(director_ids)]

print(df_director_lookup.head())

#Add primary key columns - writer
df_writer_lookup = writers_flat
df_writer_lookup = df_writer_lookup.reset_index()
df_writer_lookup = df_writer_lookup.reset_index()

#Limit to only titles of interest
df_writer_lookup = df_writer_lookup.loc[df_writer_lookup['tconst'].isin(title_ids)]

#Limit to only writers of interest
df_writer_lookup = df_writer_lookup.loc[df_writer_lookup['writers'].isin(writer_ids)]

print(df_writer_lookup.head())

       level_0  index     tconst  directors
61837    61837  52584  tt0053567  nm0633932
67009    67009  56035  tt0057092  nm0025655
68435    68435  56880  tt0057955  nm0491603
73906    73906  60475  tt0061638  nm0122236
74226    74226  60770  tt0061944  nm0223081
        level_0  index     tconst    writers
14039     14039  10410  tt0010548  nm0073592
17615     17615  12443  tt0012619  nm0505590
127277   127277  52577  tt0053560  nm0736325
131516   131516  54034  tt0055042  nm1318749
131518   131518  54034  tt0055042  nm1320539


In [5]:
#Strip down to necessary columns
df_director_lookup = df_director_lookup[['level_0','tconst','directors']]
df_writer_lookup = df_writer_lookup[['level_0','tconst','writers']]

#Rename columns
df_director_lookup.columns = ['director_lookup_id', 'title_id', 'director_id']
df_writer_lookup.columns = ['writer_lookup_id','title_id', 'writer_id']
print(df_director_lookup.head())
print(df_writer_lookup.head())

       director_lookup_id   title_id director_id
10889               10889  tt0010323   nm0927468
14180               14180  tt0013442   nm0003638
16151               16151  tt0015324   nm0000036
16504               16504  tt0015648   nm0001178
18084               18084  tt0017136   nm0000485
       writer_lookup_id   title_id  writer_id
418                 418  tt0000417  nm0894523
419                 419  tt0000417  nm0920229
13663             13663  tt0010323  nm0562346
13664             13664  tt0010323  nm0417917
17133             17133  tt0012349  nm0000122


In [6]:
#Output to csv, omitting the index column
df_director_lookup.to_csv('app_data/director_lookup.csv', index=False)
df_writer_lookup.to_csv('app_data/writer_lookup.csv', index=False)

In [7]:
#Print column headers for manual population in MySQL Workbench
print(list(df_director_lookup))
print(list(df_writer_lookup))

['director_lookup_id', 'title_id', 'director_id']
['writer_lookup_id', 'title_id', 'writer_id']
